In [9]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from keras.losses import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
sns.set_style('darkgrid')
plt.rcParams['figure.dpi'] = 108



In [10]:
dt = pd.read_csv("model_data/data.csv",sep ="\t")
dt.reset_index()
y = dt.Real
test_dt =pd.read_csv("model_data/test_data.csv",sep ="\t")
dt = dt.drop('Real',axis =1)
len_dt = len(dt)

In [11]:
frames = [dt, test_dt]
 
train_test = pd.concat(frames)

In [12]:
# standard
scaler = StandardScaler()
scaler.fit(train_test)
train_test_standard= scaler.transform(train_test)
#dt = scaler.transform(dt)
#test_dt = scaler.transform(test_dt)
ds = train_test_standard[:len_dt]
test_dt =train_test_standard[len_dt:]
#dt['Real'] =dt.apply(lambda x: 0 if x.Real ==-1 else 1, axis =1)


In [13]:
#ds =dt.drop('Real',axis=1)
print(len_dt)
#print(train_test_standard)
print(ds)
print(len(ds))
print(y.tail())

print(test_dt)

7955
[[-0.11048675 -0.43852401  0.28149735  0.12572772 -0.13898814]
 [ 0.03533805  0.30677984 -0.27427674 -0.08039888 -0.13898814]
 [-0.11135786 -0.43852401 -0.31333114 -0.08039888 -0.12762523]
 ...
 [-0.0981169  -0.20183968 -0.16011774  0.16320528 -0.13898814]
 [-0.10883163 -0.32269976 -0.29831022 -0.00544375 -0.13898814]
 [-0.11249031  0.20102727 -0.31032696 -0.08039888 -0.12762523]]
7955
7950    1
7951   -1
7952    1
7953    1
7954   -1
Name: Real, dtype: int64
[[-0.11074808 -0.44355984 -0.22921398 -0.08039888 -0.12762523]
 [-0.1063925  -0.39320148 -0.28929767 -0.08039888 -0.13141287]
 [-0.09471955 -0.05580041 -0.26526419 -0.08039888 -0.1352005 ]
 ...
 [-0.11022541 -0.42845233 -0.31032696 -0.08039888 -0.13141287]
 [-0.10769918 -0.4234165  -0.30732277 -0.08039888 -0.13141287]
 [-0.0913222  -0.40830899 -0.31032696 -0.08039888 -0.13141287]]


In [14]:
poly =PolynomialFeatures(2)
ds=poly.fit_transform(ds)
test_dt = poly.fit_transform(test_dt)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ds, y, test_size=0.2, random_state=1)
print("Training set has {} instances. Test set has {} instances.".format(X_train.shape[0], X_test.shape[0]))



Training set has 6364 instances. Test set has 1591 instances.


In [19]:
from sklearn.linear_model import RidgeCV
ridgecv = RidgeCV(alphas=[0.00387,0.00385,0.00386,0.005,0.0042,0.01,0.0045])
ridgecv.fit(X_train, y_train)
a= ridgecv.alpha_ 
ridgecv.alpha_ 

0.0042

In [28]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.01)
ridge.fit(X_train, y_train)

Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [29]:
print (ridge.coef_)
print (ridge.intercept_)


[ 0.00000000e+00 -1.02665108e+00 -7.13784886e-02  1.50713125e-01
  2.04757503e+00 -1.78768309e+02  3.10745764e-03 -6.51664603e-03
  2.79961282e-01 -4.45687257e-02 -7.87177481e+00  5.58769941e-03
  1.17046910e-02 -8.10867632e-03  5.97695815e-03 -1.04100822e-02
 -2.65575178e-02 -1.59736982e-04 -2.95891820e-03  1.37558507e+01
  2.45093630e+01]
-24.522174921977985


In [30]:
ridge.predict(X_test)

array([ 1.02571389, -1.3328108 ,  0.7650074 , ...,  0.73692011,
       -0.62917052,  0.78246999])

In [31]:
print(y_test)

3223    1
6133   -1
5856    1
3922    1
7409   -1
       ..
5706    1
1317    1
6983    1
7915   -1
4733    1
Name: Real, Length: 1591, dtype: int64


In [32]:
ridge.score(X_test, y_test)

0.860852973046809

In [33]:
y_real_test = ridge.predict(test_dt)
print(y_real_test)

y_test_result = list(map(lambda x: 1 / (1 + math.exp(-x)),y_real_test))
print(y_test_result)
result = pd.read_csv('data/sample.csv',sep = ',')
print(result.head())
result['Predicted'] = y_test_result
result.to_csv('predictions/result3.csv',sep = ',',index =False)

[-1.32383025 -0.63337735  0.04630898 ... -0.63293265 -0.63314378
 -0.63591465]
[0.21018174559725783, 0.34674512953140546, 0.5115751759282976, 0.5178831497180518, 0.35060617305458064, 0.20830362065669378, 0.5061270450893584, 0.3244482934719524, 0.3375678988521479, 0.35175716247383076, 0.3330912138150709, 0.3396905968465247, 0.5096058349117755, 0.2732005626434315, 0.34895329231206884, 0.5770191454133937, 0.5230293090168651, 0.2087606327093578, 0.3489706303215904, 0.5114968003253507, 0.11765374493131177, 0.6034518940487811, 0.3424303916920889, 0.20832917522273586, 0.5146297520948452, 0.3470425388507423, 0.2185731612871921, 0.3643436680682991, 0.36196194809265647, 0.33871878169289343, 0.21104474809946264, 0.5605111533480487, 0.3474321603582322, 0.35679898297067, 0.5163511058274016, 0.6086019516230413, 0.5321071336621543, 0.5169742635718206, 0.49233896178057357, 0.515464858803932, 0.3469297581459803, 0.5184680785912372, 0.5134100419186756, 0.5176383455857538, 0.34524556412465335, 0.34779177

In [34]:

median = np.median(y_test_result)
print(median)

0.4944622396635717


In [35]:
from sklearn.linear_model import Ridge
X_train = X_train/ 100.0
X_test = X_test / 100.0

In [11]:
degrees = list(range(12))
models = list()
train_mses = list()
test_mses = list()

X_grid = np.linspace(min(X_train.min(), X_test.min()),
                     max(X_train.max(), X_test.max()), num=1001)

plt.figure(figsize=(20,16))
for i, degree in enumerate(degrees):
    plt.subplot(len(degrees)//2, 2, i+1)

    # Transform features
    poly = PolynomialFeatures(degree=degree)
    Phi_train, Phi_test = poly.fit_transform(X_train), poly.fit_transform(X_test)
    Phi_grid = poly.fit_transform(X_grid[:,np.newaxis])

    # Fit model
    lr_poly = Ridge(alpha = 0.002).fit(Phi_train, y_train)
    models.append(lr_poly)
    
    # Evaluate
    train_mse = mean_squared_error(lr_poly.predict(Phi_train), y_train)
    train_mses.append(train_mse)
    test_mse = mean_squared_error(lr_poly.predict(Phi_test), y_test)
    test_mses.append(test_mse)

    plt.plot(X_grid, lr_poly.predict(Phi_grid), 'k', label='Prediction')
    plt.scatter(X_train, y_train, color='b', marker='.', label='Train')
    #plt.scatter(X_test_s, y_test, color='r', marker='.', label='Test')
    plt.title('Degree {} | Train MSE {:.3f}'.format(degree, train_mse))
    plt.legend()

plt.suptitle('Polynomial ridge regression for different polynomial degrees', y=1.05, fontsize=32)
plt.tight_layout()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().